## Start of project

In [ ]:
!pip install -U spacy
!pip install -U matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.4 MB 9.0 MB/s 
     |████████████████████████████████| 965 kB 63.6 MB/s 
     |████████████████████████████████| 295 kB 74.9 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
!pip show matplotlib

Name: matplotlib
Version: 3.6.2
Summary: Python plotting package
Home-page: https://matplotlib.org
Author: John D. Hunter, Michael Droettboom
Author-email: matplotlib-users@python.org
License: PSF
Location: /usr/local/lib/python3.8/dist-packages
Requires: pillow, cycler, python-dateutil, packaging, numpy, contourpy, pyparsing, fonttools, kiwisolver
Required-by: yellowbrick, wordcloud, seaborn, scikit-image, pycocotools, prophet, plotnine, pandas-profiling, mlxtend, mizani, missingno, matplotlib-venn, keras-vis, imgaug, fastai, descartes, datascience, daft, arviz


In [ ]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 17 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import random
random.seed(12)
import spacy
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## **PART 1: LSTM FOR LANGUAGE MODELING**

**Task 1:** Follow the tutorial in [here](https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/) to train a word-level LSTM language model. Train the language model on texts
from the file prideAndPrejudice.txt. Before using it to train the language model, you need to first sentence-segment, then
tokenize, then lower case each line of the file using Spacy. Append start-of-sentence token ’\<s\>’ and end-of-sentence
’\</s\>’ token to each sentence and put each sentence in its own line. Use only words that appear more than once in this
corpus and assign UNK tokens for the rest; you may also need to pad sentences that are shorter than 5 (see [here](https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb) in cell
10-12 for adding unknown: UNK token and padding: PAD token to your vocabulary). Train the language model and
save the trained model (see [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html)). Generate 10 examples of text from it, starting from ’\<s\>’ token and ending at ’\</s\>’
token.

**Checkpoint 1:**

Load texts from by lines. The samples of your loaded text lines should look like the following.

In [ ]:
def findFiles(path): return glob.glob(path)

# Build the category_lines dictionary, a list of names per language
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [line for line in lines]

for filename in findFiles('/content/drive/MyDrive/Colab Notebooks/CS_505_PA6/pride*.txt'):
    novel = os.path.splitext(os.path.basename(filename))
    all_categories.append(novel)
    lines = readLines(filename)


In [ ]:
print([lines[695],lines[0]])

n = 5
random.sample(lines, n)

['"It is evident by this," added Jane, "that he comes back no more this winter."', 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.']


['"I knew," said he, "that what I wrote must give you pain, but it was necessary. I hope you have destroyed the letter. There was one part especially, the opening of it, which I should dread your having the power of reading again. I can remember some expressions which might justly make you hate me."',
 '"I am much obliged to your ladyship for your kind invitation," replied Elizabeth, "but it is not in my power to accept it. I must be in town next Saturday."',
 '"Do not give way to useless alarm," added he; "though it is right to be prepared for the worst, there is no occasion to look on it as certain. It is not quite a week since they left Brighton. In a few days more we may gain some news of them; and till we know that they are not married, and have no design of marrying, do not let us give the matter over as lost. As soon as I get to town I shall go to my brother, and make him come home with me to Gracechurch Street; and then we may consult together as to what is to be done."',
 '"Bo

In [ ]:
lines_dict = dict(enumerate(lines))
print(lines_dict)
print(len(lines_dict))


{0: 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.', 1: 'However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.', 2: '"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"', 3: 'Mr. Bennet replied that he had not.', 4: '"But it is," returned she; "for Mrs. Long has just been here, and she told me all about it."', 5: 'Mr. Bennet made no answer.', 6: '"Do you not want to know who has taken it?" cried his wife impatiently.', 7: '"_You_ want to tell me, and I have no objection to hearing it."', 8: 'This was invitation enough.', 9: '"Why, my dear, you must know, Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England; that he 

**Checkpoint 2:**

Preprocess your loaded texts with spacy, also convert words into lower-case, and also pad sentences with \<s\> and \</s\>. The preprocessed lines should look like the following.

In [ ]:
def preprocess(textList, batchsz=500):
    # Input: text separated by new lines
    # Output: sentences with space around punctuation

    # sub-task 1: segment all the sentences in the wiki texts.
    # sub-task 2: tokenize the sentences from sub-task 2.
    # sub-task 3: lower-case the tokens from sub-task 3/4.

    nlp = spacy.load('en_core_web_lg')
    docDict = {}
    docList = []
    dropped = []
    cnt = 0

    for doc in tqdm(nlp.pipe(textList, batch_size=batchsz)): # Step 1
        # if doc.has_annotation("DEP"):
        # for ent in doc.ents:
            #  THIS LINE GIVES THE NAMED ENTITIES IN DOC
            # print(ent.text, ent.start_char, ent.end_char, ent.label_)
        for spanText in doc.sents:
            spanT = spanText.text.strip()
            if len(spanT) < 8:
                dropped.append(spanT)
                continue
            if spanT:
                docString = '<s> ' 
                for tok in spanText:
                    docString += tok.text.strip().lower() + ' ' # Step 2 & 3
                docString += ' </s>'
                docDict[cnt]= docString
                docList.append(docString)
                cnt += 1
    return  docDict, docList, dropped

In [ ]:
sentDict, sent_list, dropped = preprocess(lines)
print(f" \nline size: {len(sentDict)}")
print(sent_list[0:2])

2063it [00:19, 108.14it/s]

 
line size: 6005
['<s> it is a truth universally acknowledged , that a single man in possession of a good fortune , must be in want of a wife .  </s>', '<s> however little known the feelings or views of such a man may be on his first entering a neighbourhood , this truth is so well fixed in the minds of the surrounding families , that he is considered the rightful property of some one or other of their daughters .  </s>']


**Checkpoint 3:**

Define a function to prepare fixed-length sequences from our dataset. Here we specified the length of the sequence as five. You may refer to the tutorial to see how this can be done. Remember to deal with the case of tokens of length less or equal to 5. The following example show what the fixed-length sequence should look like given  input texts. Also compare the size of the processed sequences to see if the number is close.

NOTE: the example is just the processed result of one line, your result sequences should contain processed results for all lines.

In [ ]:
# create sequences of length 5 tokens
def create_seq(text, seq_len = 5):
    
    sequences = []

    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
        for i in range(seq_len, len(text.split())):
            # select sequence of tokens
            seq = text.split()[i-seq_len:i+1]
            # add to the list
            sequences.append(" ".join(seq))

        return sequences

    # if the number of tokens in 'text' is less than or equal to 5
    else:
        # select sequence of tokens
        seq = text.split()
        for i in range(5-len(seq)+1):
            seq.append('_PAD')
        # add padding to the list to fill out all 5 positions
        sequences.append(" ".join(seq))

        return sequences

In [ ]:
# sample the original sentence
print(sent_list[123])

seqs = [create_seq(i) for i in sent_list if i != '_']

# sample the sentence separated text
print(seqs[123])

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
print(f'lines size: {len(sent_list)} \nseq size: {len(seqs)}')

<s> in a few days mr. bingley returned mr. bennet 's visit , and sat about ten minutes with him in his library .  </s>
['<s> in a few days mr.', 'in a few days mr. bingley', 'a few days mr. bingley returned', 'few days mr. bingley returned mr.', 'days mr. bingley returned mr. bennet', "mr. bingley returned mr. bennet 's", "bingley returned mr. bennet 's visit", "returned mr. bennet 's visit ,", "mr. bennet 's visit , and", "bennet 's visit , and sat", "'s visit , and sat about", 'visit , and sat about ten', ', and sat about ten minutes', 'and sat about ten minutes with', 'sat about ten minutes with him', 'about ten minutes with him in', 'ten minutes with him in his', 'minutes with him in his library', 'with him in his library .', 'him in his library . </s>']
lines size: 6005 
seq size: 126413


**Checkpoint 4:**

Create inputs and targets (x and y) for training; a sample of your x and y should look like the following. Remember to pad sequences with less than 5 tokens. (NOTE: the sample is just a pair of x,y, not all x's /y's for inputs and targets).

In [ ]:
# create inputs and targets (x and y)
x = []
y = []

for s in seqs:
    x.append(" ".join(s.split()[:-1]))
    y.append(" ".join(s.split()[1:]))

# print(f'sequence: {seqs[12]}')
print(f'sample x: {x[12]}')
print(f'sample y: {y[12]}')

sample x: in possession of a good
sample y: possession of a good fortune


**Checkpoint 5:**

Compute a list of all words that need to be assigned as UNK. Part of the unknown list should look like the following.

In [ ]:
# create integer-to-token mapping
int2token = {}
tok_cnt = 0
singleUNK = []

tokList = " ".join(sent_list).split()
for w in set(" ".join(sent_list).split()):
    if tokList.count(w) > 1:
        int2token[tok_cnt] = w
        tok_cnt+= 1
    else:
        singleUNK.append(w)

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

set_singleUNK = set(singleUNK)
print(f' # of singletons {len(singleUNK)}')
# testing to match the provided answer
  # test original sentence list for count
print(f' Test for singleUNK  defended {" ".join(sent_list).split().count("defended")==1}')
  # test identified singleUNK tokens
print(f" Test for singleUNK  unwearying {singleUNK.count('unwearying')==1}, although {singleUNK.count('although') == 1}")
# list of single words assigned to UNK
print(singleUNK)

 # of singletons 2446
 Test for singleUNK  defended True
 Test for singleUNK  unwearying True, although True
['tricks', '15th', 'stress', 'diffident', 'postponed', 'exceeding', 'impute', 'sleepless', 'defective', 'substitute', 'songs', 'portrait', 'giddiness', 'inexhaustible', 'disbelieving', 'manoeuvre', 'hardship', 'awaited', 'prepossessed', 'uproar', 'sentences', 'imitated', 'equals', 'tries', 'glen', 'feet', 'he?--poor', 'exclusion', 'overhearings', 'foxhounds', 'man!--so', 'disrespectful', 'indefinite', 'secluded', 'fame', 'disquiet', 'arch', 'obligingly', 'hire', 'overjoyed', 'heedless', 'colder', 'elevating', 'ragout', 'flutterings', 'demanding', 'apace', 'sleeping', 'west', 'prejudiced', 'stupider', 'succession', 'bets', 'savours', 'did_--i', 'sportive', 'trembled', 'blow', 'chanced', 'precluding', 'mess', 'drinking', 'retains', 'immoral', 'obeisance', 'conditional', 'scrape', 'threats', 'simply', 'grapes', 'unblemished', 'intimation', 'dissemble', 'mince', 'abhorrent', 'footst

['unwearying', 'defended', 'pointedly', 'although', 'utterly', 'resisted', 'gratification', 'chimney', 'digressions', 'improvements', 'retail', 'imitations', 'china', 'broad', 'faced', 'stuffy', 'breathing', 'port', 'commonest', 'dullest', 'topic', 'speaker', 'rivals', 'muffin', 'life--', 'compliance', 'engrossing', 'talker', 'bets', 'prizes', 'demands', 'hesitating', 'anywhere', 'estimated', 'imposing', 'proclaim', 'breathed', 'truest', 'verily', 'topics', 'military', 'bequeathed', 'gift', 'godfather', 'heavens', 'disregarded', 'legal', 'redress', 'informality', 'bequest', 'conditional', 'forfeited', 'disgraced', 'irritated', 'inhumanity', 'boasting', 'implacability', 'resentments', 'amiable"--but', 'childhood', 'inmates', 'parental', 'confidential', 'superintendence', 'providing', 'dishonest', 'dishonesty', 'impulses', 'degenerate', 'popular', 'guardian', 'superintends', 'trials', 'reverting', 'conversible', 'equals', 'chances', 'shillings', 'regarding', 'misleads', 'arrogant', 'dict

**Checkpoint 6:**

Build two dictionaries (vocabulary), one maps a unique integer to a unique token. The other one is a mapping from the reversed direction. The following is an example of the key-value pairs of the two dictionaries. Your dictionaries should have similar formats.

In [ ]:
print(f"key: 'the' value: {token2int['the']}")
print(f"key: {int2token[1244]} value: 1244")
print(f"size of the dictionaries (vocabulary): {len(int2token)} and {len(token2int)}")

key: 'the' value: 1995
key: settled value: 1244
size of the dictionaries (vocabulary): 3905 and 3905


key: 'the' value:  1502
key: 1327, value:  the
size of the dictionaries (vocabulary): 3907 and 3907


The difference in the dictionary of 3905 items to one with 3907 items is:

Not in the reference dictionary: \*, la
Below is the cell that provides my proof.  

Sentences dropped from the book: {'"La!', '"Ah!', 'Oh!', '"If!', 'Lord!', 'Well!', '"All!', ';', "'", '"Oh!', '"True.', '"Yes.', '"Money!', '_', '*', 'no."', 'Adieu!', 'no.', 'oh!', 'so.', '* *', 'yes.', ',', 'Ah!', '"_That_', '"EDW.', 'That _'}


** I captured the sentences in the output provided as correct results in the ipynb to determine the words in the token dictionary. 

In [ ]:
save_3905 = int2token.values()
save_3907 = int2token.values()

In [ ]:


print(f'Not in the reference dictionary:')
for i in save_3907:
    if i not in set(save_3905):
        print(f'{i}')

print(f'Sentences dropped from the book: {set(dropped)}')

Not in the reference dictionary:
la
*
Sentences dropped from the book: {'"Ah!', '* *', 'so.', '"True.', '"_That_', 'oh!', '*', 'Lord!', 'no.', 'That _', '"Money!', '"Yes.', '"All!', ',', '"Oh!', 'Ah!', 'yes.', '"EDW.', '"If!', 'Adieu!', '_', 'Oh!', "'", 'Well!', ';', '"La!', 'no."'}


**Checkpoint 7:**

Add unknown and padding to the two dictionaries (check). The size of them should increase by 2.

In [ ]:
# add unknown and padding tokens
int2token[tok_cnt] = '_UNK'
tok_cnt+=1
int2token[tok_cnt] = '_PAD'

# re-create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

print(f"vocabulary sizes: {len(int2token)},{len(token2int)}")
print(f"token2int['_PAD'] {token2int['_PAD']}")
print(f"token2int['_UNK'] {token2int['_UNK']}")
vocab_size = len(int2token)

vocabulary sizes: 3907,3907
token2int['_PAD'] 3906
token2int['_UNK'] 3905


vocabulary sizes: 3909,3909
token2int['_PAD'] 3908
token2int['_UNK'] 3907


**Checkpoint 8:**

Convert your sequences of tokens into integers, with the dictionaries we just built. Your x's and y's should look like the following.

In [ ]:
def get_integer_seq(seq):
  return [token2int[w] if w not in set_singleUNK else token2int['_UNK'] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

def get_batches(arr_x, arr_y, batch_size):

    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
      x = arr_x[prv:n,:]
      y = arr_y[prv:n,:]
      prv = n
      yield x, y

In [ ]:
print(f'x in integers: {x_int[0:6]}')
print(f'y in integers: {y_int[0:6]}')

x in integers: [[2126 3607 3187  423 2361]
 [3607 3187  423 2361 2773]
 [3187  423 2361 2773 2573]
 [ 423 2361 2773 2573 1206]
 [2361 2773 2573 1206 2578]
 [2773 2573 1206 2578  423]]
y in integers: [[3607 3187  423 2361 2773]
 [3187  423 2361 2773 2573]
 [ 423 2361 2773 2573 1206]
 [2361 2773 2573 1206 2578]
 [2773 2573 1206 2578  423]
 [2573 1206 2578  423 3787]]


x in integers: 
[[2514 3142 1934 1341 3284]
 [3142 1934 1341 3284 2482]
 [1934 1341 3284 2482  645]
 ...
 [1002 1502 1824  148 3907]
 [1502 1824  148 3907  533]
 [1824  148 3907  533  322]]

y in integers: 
[[3142 1934 1341 3284 2482]
 [1934 1341 3284 2482  645]
 [1341 3284 2482  645 3167]
 ...
 [1502 1824  148 3907  533]
 [1824  148 3907  533  322]
 [ 148 3907  533  322 1356]]


**Checkpoint 9:**

Build your LSTM model with Pytorch. Your LSTM structure should look like the following:



In [ ]:
class WordLSTM(nn.Module):

    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)

    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''
       ## pass input through embedding layer
        embedded = self.emb_layer(x)

        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)

        ## pass through a dropout layer
        out = self.dropout(lstm_output)

        #out = out.contiguous().view(-1, self.n_hidden)
        out = out.reshape(-1, self.n_hidden)

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden


    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available   mps is for mac code
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())

        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

In [ ]:
# instantiate the model
net = WordLSTM()

# push the model to GPU (avoid it if you are not using the GPU)
dtype = torch.float
device = torch.device("cuda")
net.cuda()

print(net)

WordLSTM(
  (emb_layer): Embedding(3907, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=3907, bias=True)
)


WordLSTM(
  (emb_layer): Embedding(3909, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=3909, bias=True)
)


**Checkpoint 10:**

Train your model with the processed data.

In [ ]:
def train(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):

    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)

    # loss
    criterion = nn.CrossEntropyLoss()

    # push model to GPU
    net.cuda()

    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1

            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()

            if counter % print_every == 0:

              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [ ]:
# train the model
train(net, batch_size = 32, epochs=50, print_every=256)

Epoch: 1/50... Step: 256...
Epoch: 1/50... Step: 512...
Epoch: 1/50... Step: 768...
Epoch: 1/50... Step: 1024...
Epoch: 1/50... Step: 1280...
Epoch: 1/50... Step: 1536...
Epoch: 1/50... Step: 1792...
Epoch: 1/50... Step: 2048...
Epoch: 1/50... Step: 2304...
Epoch: 1/50... Step: 2560...
Epoch: 1/50... Step: 2816...
Epoch: 1/50... Step: 3072...
Epoch: 1/50... Step: 3328...
Epoch: 1/50... Step: 3584...
Epoch: 1/50... Step: 3840...
Epoch: 2/50... Step: 4096...
Epoch: 2/50... Step: 4352...
Epoch: 2/50... Step: 4608...
Epoch: 2/50... Step: 4864...
Epoch: 2/50... Step: 5120...
Epoch: 2/50... Step: 5376...
Epoch: 2/50... Step: 5632...
Epoch: 2/50... Step: 5888...
Epoch: 2/50... Step: 6144...
Epoch: 2/50... Step: 6400...
Epoch: 2/50... Step: 6656...
Epoch: 2/50... Step: 6912...
Epoch: 2/50... Step: 7168...
Epoch: 2/50... Step: 7424...
Epoch: 2/50... Step: 7680...
Epoch: 3/50... Step: 7936...
Epoch: 3/50... Step: 8192...
Epoch: 3/50... Step: 8448...
Epoch: 3/50... Step: 8704...
Epoch: 3/50... St

**Checkpoint 11:**

Generate 10 examples of text from the model, starting from ’\<s\>’ token and ending at ’\</s\>’ token. The generated texts should look more or less natural, instead of being in a very fixed pattern or randomly generated. Some examples are shown below.

In [ ]:
# predict next token
def predict(net, tkn, h=None):
        
  print(tkn)
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]
  print(len(p))
  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h


# function to generate text
def sample(net, size, prime='it is'):
        
    # push to GPU
    net.cuda()
    
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
        token, h = predict(net, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(net, toks[-1], h)
        toks.append(token)
        if token == '</s>':
            break

    return ' '.join(toks)

In [ ]:
for i in range(11):
    print(sample(net,1105, prime='<s>'))

NameError: ignored

**Task 2:**

Compute and report the perplexity of the saved model on test_1.txt and test_2.txt files. Note that the test files are already preprocessed. Compute the complexity of the test_2.txt by saving the model first and then loading it.

**Checkpoint 1:**

Load your test_1.txt file in lines as you did in task 1. The results should look similar to the following.

In [ ]:
def return_lines(fname):
    for filename in findFiles('/content/drive/MyDrive/Colab Notebooks/CS_505_PA6/'+ fname):
        novel = os.path.splitext(os.path.basename(filename))
        all_categories.append(novel)
        o_lines = readLines(filename)
    return o_lines

t1_lines = return_lines('test_1.txt')
t2_lines = return_lines('test_2.txt')

In [ ]:
print('TEST ONE')
print(random.sample(t1_lines, n, ))

print('TEST TWO')
print(random.sample(t2_lines, n))

TEST ONE
['<s> it is a very large one , i know ; and when i come to see you , i hope you will have new - furnished it , for it wanted it very much when i was there six years ago . " </s>', '<s> to marianne , he merely bowed and said nothing . </s>', '<s> i am sure you must have heard of her before . </s>', '<s> they gave themselves up wholly to their sorrow , seeking increase of wretchedness in every reflection that could afford it , and resolved against ever admitting consolation in future . </s>', '<s> " it is enough , " said she ; " to say that he is unlike fanny is enough . </s>']
TEST TWO
['<s> @nickkkjonasss i like poison ivy the best </s>', '<s> " has 0 g of milky bar left , and around 200ml of coke " </s>', '<s> " so rylee , grace ... wana go steve \'s party or not ? ? sadly since its easter i wnt b able 2 do much   but ohh well ..... " </s>', "<s> http://xlisaatown.myminicity.com/   & lt;3 now i 'm tha bomb </s>", '<s> " @cliff_forster yeah , that does work better than just wa

['<s> the family of dashwood had long been settled in sussex . </s>', '<s> their estate was large , and their residence was at norland park , in the centre of their property , where , for many generations , they had lived in so respectable a manner as to engage the general good opinion of their surrounding acquaintance . </s>', '<s> the late owner of this estate was a single man , who lived to a very advanced age , and who for many years of his life , had a constant companion and housekeeper in his sister . </s>', '<s> but her death , which happened ten years before his own , produced a great alteration in his home ; for to supply her loss , he invited and received into his house the family of his nephew mr. henry dashwood , the legal inheritor of the norland estate , and the person to whom he intended to bequeath it . </s>', "<s> in the society of his nephew and niece , and their children , the old gentleman 's days were comfortably spent . </s>", '<s> his attachment to them all incre

**Checkpoint 2:**

Compute the perplexity on test_1.txt. Your final perplexity should look like the one below. (You may need to implement the perplexity calculation on your own instead of using any existing ones from libraries.)

In [ ]:
# create a dict of all words in test_1 tok and int

# create a list of unk words in test  map to _UNK

# send predict every word in test set and retrieve next word for actual next word token #

# add probabilities together for all words divide by N (vocab len including toks) and log 2.


# predict next token
def pp_next(net, tkn, target_tok, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], havg_perp=0
len_test = 0
while True:
    try:
        t = next(test)
    except StopIteration:
        break
    perp_score = model1.perplexity(t)
    avg_perp+= perp_score
    len_test +=1

    # print(' '.join(test_nft_list_1f[0]))
    # print(perp_score)
    # temp = test_nft_list_1f.pop(0)

perplexity on test_1.txt is:  172.75867345155558


**Checkpoint 3:**

Save your model with the 'state_dict()' method. Load your saved model and sample some generated texts to see if it works properly. The sample texts could look like the following one.

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './net_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving network to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
net_to_save = net.module if hasattr(net, 'module') else net  # Take care of distributed/parallel training
net_to_save.state_dict(output_dir)  #  ???
# tokenizer.state_dict(output_dir)

# Good practice: save your training arguments together with the trained model
torch.save(args, os.path.join(output_dir, 'training_args.bin'))

In [ ]:
!ls -l --block-size=K ./net_save/

In [ ]:
!ls -l --block-size=M ./net_save/pytorch_model.bin

In [ ]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./net_save/ "./PA7_BERT Fine-Tuning/"

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = './net_save/'
net = WordLSTM(output_dir)
# tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
net.cuda()

In [ ]:
for i in range(11):
    print(sample(net,1105, prime='<s>'))

'<s> " oh , yes ! pray be _UNK , i am afraid of her , she could not be insensible to the latter method , " said elizabeth ; and she had no reason , i am sure i have not a little mistake , and the _UNK _UNK of his _UNK , she could not help cautioning her to see her ; for she was not in a very different _UNK . " </s>'

**Checkpoint 4:**

Load your test_2.txt file in lines. Compute the perplexity with your loaded model. The perplexity should look close to (or better than) the following one.

In [ ]:
# run perplexity on test_2

perplexity on test_2.txt is:  951.2221688812576


**Task 3:**

Train the language model as before, but with input sequence lengths of 25 (currently, its inputs are of length 5).
You may need to pad some of the shorter sentences to length 25. Save your trained model. Generate 10 examples of text
from it, starting from ’\<s\>’ token and ending at ’\</s\>’ token. **Answer in a separate cell:** Are there differences from the generated examples from
2.1? 

**Task 4:**
Compute and report the perplexity of this saved model on test 1.txt file. **Answer in a separate cell:** Is there any reason why this model is better/worse than the previous one (length = 5)?

NOTE: at this point you should be able to finish the tasks without the checkpoints, try to re-use the code you've implemented.

In [ ]:
# change to 25 and rerun

In [ ]:
# run perplexity on new model

**Task 5:**

Train the better language model as before but start with pre-trained Glove6B 100d embeddings (see [here](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76) on how
to incorporate pretrained embeddings in your LSTM model). This time, use all your words in the corpus as vocabulary,
even those occurring only once in the corpus. Only assign UNK token to words that are not in the Glove vocabulary and
initialize random vectors in the embedding matrix for the UNK, \<s\>, \</s\>, and PAD tokens using a Gaussian
distribution with mean 0.0 and σ set to 0.6 (see numpy.random.normal). Save your trained model. Generate 10 examples of text from it,
starting from the ’\<s\>’ token and ending at the ’\</s\>’ token. Are there differences from the generated examples from before?

**Task 6:**
Compute and report the perplexity of this saved model on the test 1.txt file.

NOTE: 

1. The biggest difference between this task and the previous starts from the word-integer dictionary. In addition to the mappings between word and integer, you also need to have a dictionary that maps the word to the word vector in Glove. Don't forget to add UNK, PAD, \<s\> and \</s\> to the dictionary. As these tokens do not exist in Glove, you need np.random.normal(...) to assign them random values.
2. The next difference is since the most of the word embeddings are pre-trained, you need to initialize nn.Embed with the Glove embedding matrix. You also need to adjust the model to fit the embedding as well.
3. The training might take a very long time, make sure you have your GPU utilized for Pytorch and get enough time running it.

In [ ]:
# change embedding

In [ ]:
# run perplexity